In [1]:
import logging
import sys
import os

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from dotenv import load_dotenv
load_dotenv()

True

In [1]:
import pandas as pd

df = pd.read_csv("data/판교맛집_info.csv")
df.head()

,id,name,category,address,amenities,operations,link,menuLink
0,35255743,도원참치 판교본점,생선회,경기 성남시 분당구 판교역로192번길 14-2 골드타워 2층 도원참치,"포장, 배달, 무선 인터넷, 남/녀 화장실 구분, 단체 이용 가능, 예약, 유아의자...",22:00에 라스트오더,https://m.place.naver.com/restaurant/35255743/...,https://m.place.naver.com/restaurant/35255743/...
1,1880745564,닥터로빈 시그니처 판교점,이탈리아음식,경기 성남시 분당구 동판교로177번길 25 아브뉴프랑 1층 108호 닥터로빈,"포장, 배달, 예약, 무선 인터넷, 남/녀 화장실 구분, 주차",21:00에 라스트오더,https://m.place.naver.com/restaurant/188074556...,https://m.place.naver.com/restaurant/188074556...
2,1284328078,이가네양갈비 판교점,양갈비,경기 성남시 분당구 판교역로 145,"단체 이용 가능, 주차, 남/녀 화장실 구분, 예약",10:45에 영업 종료,https://m.place.naver.com/restaurant/128432807...,https://m.place.naver.com/restaurant/128432807...
3,1765758635,오늘와인한잔 야탑점,와인,경기 성남시 분당구 장미로100번길 29 1층 오늘와인한잔,"남/녀 화장실 구분, 포장",17:00에 영업 시작,https://m.place.naver.com/restaurant/176575863...,https://m.booking.naver.com/order/bizes/457246...
4,1948630584,카츠쇼신 판교점,돈가스,경기 성남시 분당구 동판교로177번길 25 2층 204호,"남/녀 화장실 구분, 유아의자, 간편결제, 주차, 발렛파킹",15:00에 브레이크타임,https://m.place.naver.com/restaurant/194863058...,https://m.place.naver.com/restaurant/194863058...


In [2]:
df = pd.read_csv("data/판교맛집_menus.csv")
df.head()

,resturant_id,name,description,price,menuLink,menu_id
0,35255743,일품(1인무한리필),NaN,59000,NaN,1
1,35255743,대진참치(1인무한리필),NaN,48000,NaN,2
2,35255743,혼마구로(1인무한리필),NaN,85000,NaN,3
3,35255743,혼골드참치(1인무한리필),NaN,100000,NaN,4
4,1880745564,시그니처 통단호박콩크림스프(2~3인),NaN,23800,NaN,5


In [2]:
username="neo4j"
password="AStglmnxZLS4bYASbsGkrV8ILu3XGJDdaeKiIxdTBvs"
url="neo4j+s://6b215408.databases.neo4j.io"
database="neo4j"

# username="neo4j"
# password="qwer1234"
# url="bolt://localhost:7687"
# database="neo4j"

In [1]:
# DB credentials
username="neo4j"
password="qwer1234"
url="bolt://localhost:7687"
# database="pangyo-restaurant"
database="sample-jobs"

In [2]:
from langchain.chains import GraphCypherQAChain
from langchain_community.graphs import Neo4jGraph
from langchain_openai import ChatOpenAI

graph = Neo4jGraph(
    url=url, 
    username=username, 
    password=password,
    database=database,
)

In [3]:
graph.refresh_schema()
print(graph.schema)

Node properties are the following:
Job {name: STRING, content: STRING},Company {address: STRING, name: STRING},CareerCondition {name: STRING},GenderCondition {name: STRING},AgeCondition {name: STRING},Contact {email: STRING, phone: STRING, name: STRING},BusinessCategory {name: STRING},WorkType {name: STRING},Payment {type: STRING, value: INTEGER, name: STRING}
Relationship properties are the following:
IS_ONE_OF_CONDITIONS {type: STRING},IS_TYPE_OF {type: STRING}
The relationships are the following:
(:Company)-[:HAS]->(:Job),(:CareerCondition)-[:IS_ONE_OF_CONDITIONS]->(:Job),(:GenderCondition)-[:IS_ONE_OF_CONDITIONS]->(:Job),(:AgeCondition)-[:IS_ONE_OF_CONDITIONS]->(:Job),(:Contact)-[:REPRESENT]->(:Company),(:BusinessCategory)-[:IS_TYPE_OF]->(:Job),(:WorkType)-[:IS_TYPE_OF]->(:Job),(:Payment)-[:IS_PAYMENT_FOR]->(:Job)


In [17]:
movies_query = """
LOAD CSV WITH HEADERS FROM 
'https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/movies/movies_small.csv'
AS row
MERGE (m:Movie {id:row.movieId})
SET m.released = date(row.released),
    m.title = row.title,
    m.imdbRating = toFloat(row.imdbRating)
FOREACH (director in split(row.director, '|') | 
    MERGE (p:Person {name:trim(director)})
    MERGE (p)-[:DIRECTED]->(m))
FOREACH (actor in split(row.actors, '|') | 
    MERGE (p:Person {name:trim(actor)})
    MERGE (p)-[:ACTED_IN]->(m))
FOREACH (genre in split(row.genres, '|') | 
    MERGE (g:Genre {name:trim(genre)})
    MERGE (m)-[:IN_GENRE]->(g))
"""

graph.query(movies_query)

[]

In [8]:
# LOAD CSV

seeding_query = """
LOAD CSV WITH HEADERS FROM "https://raw.githubusercontent.com/kofsitho87/RAG/main/%E1%84%91%E1%85%A1%E1%86%AB%E1%84%80%E1%85%AD%E1%84%86%E1%85%A1%E1%86%BA%E1%84%8C%E1%85%B5%E1%86%B8_info.csv" 
AS row
WITH row WHERE row.id IS NOT NULL

MERGE (r:Restaurant {id:toInteger(row.id)})
SET r.name = row.name,
    r.address = row.address,
    r.phone = row.phone,
    r.operation = row.operations

MERGE (c:Category {name: trim(row.category)})
MERGE (r)-[:hasCategory]->(c)

FOREACH (amenity in split(row.amenities, ',') | 
    MERGE (s:Service {name:trim(amenity)})
    MERGE (r)-[:OFFERS]->(s))
"""

graph.query(seeding_query)

[]

In [8]:
df = pd.read_csv("data/판교맛집_menus.csv")
df.head()

,resturant_id,name,description,price,menuLink,menu_id
0,35255743,일품(1인무한리필),NaN,59000,NaN,1
1,35255743,대진참치(1인무한리필),NaN,48000,NaN,2
2,35255743,혼마구로(1인무한리필),NaN,85000,NaN,3
3,35255743,혼골드참치(1인무한리필),NaN,100000,NaN,4
4,1880745564,시그니처 통단호박콩크림스프(2~3인),NaN,23800,NaN,5


In [9]:
df[['name', 'description', 'price']][:50]

,name,description,price
0,일품(1인무한리필),NaN,59000
1,대진참치(1인무한리필),NaN,48000
2,혼마구로(1인무한리필),NaN,85000
3,혼골드참치(1인무한리필),NaN,100000
4,시그니처 통단호박콩크림스프(2~3인),NaN,23800
5,시그니처 매콤새우파스타,NaN,24000
6,통관자 바질페스토 파스타,NaN,26000
7,비프스테이크,NaN,46900
8,스모크드 뽈뽀스테이크,NaN,32000
9,올리브&올리브파스타,NaN,19800


In [13]:
# LOAD CSV

seeding_query = """
LOAD CSV WITH HEADERS FROM "https://raw.githubusercontent.com/kofsitho87/RAG/main/%E1%84%91%E1%85%A1%E1%86%AB%E1%84%80%E1%85%AD%E1%84%86%E1%85%A1%E1%86%BA%E1%84%8C%E1%85%B5%E1%86%B8_menus.csv" 
AS row
WITH row, toInteger(row.resturant_id) AS restaurantId
WHERE row.resturant_id IS NOT NULL

MERGE (m:Menu {name:row.name})
SET m.description = row.description,
    m.price = row.price

WITH row, m, restaurantId
MATCH (r:Restaurant {id: restaurantId})
MERGE (r)-[:SERVES]->(m)
MERGE (m)-[:IS_SERVED_AT]->(r)
"""

graph.query(seeding_query)

[]

In [7]:
graph.refresh_schema()
print(graph.schema)

Node properties are the following:
Job {name: STRING, content: STRING},Company {address: STRING, name: STRING},CareerCondition {name: STRING},GenderCondition {name: STRING},AgeCondition {name: STRING},Contact {email: STRING, phone: STRING, name: STRING},BusinessCategory {name: STRING},WorkType {name: STRING},PaymentType {name: STRING},PaymentValue {value: INTEGER}
Relationship properties are the following:
IS_ONE_OF_CONDITIONS {type: STRING},IS_TYPE_OF {type: STRING},IS_PAYMENT {type: STRING}
The relationships are the following:
(:Company)-[:HAS]->(:Job),(:CareerCondition)-[:IS_ONE_OF_CONDITIONS]->(:Job),(:GenderCondition)-[:IS_ONE_OF_CONDITIONS]->(:Job),(:AgeCondition)-[:IS_ONE_OF_CONDITIONS]->(:Job),(:Contact)-[:REPRESENT]->(:Company),(:BusinessCategory)-[:IS_TYPE_OF]->(:Job),(:WorkType)-[:IS_TYPE_OF]->(:Job),(:PaymentType)-[:IS_PAYMENT]->(:Job),(:PaymentValue)-[:IS_PAYMENT]->(:Job)


In [8]:
llm = ChatOpenAI(model="gpt-4", temperature=0)

chain = GraphCypherQAChain.from_llm(
    graph=graph, 
    cypher_llm=llm,
    qa_llm=llm,
    validate_cypher=True,
    verbose=True,
)

In [9]:
# chain.run("홀서빙에 관한 직업을 찾아줘")
chain.run("시급이 1만원이 넘는 직업을 찾아줘")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (j:Job)<-[r:IS_PAYMENT]-(p:PaymentValue) WHERE p.value > 10000 RETURN j.name
Full Context:
[{'j.name': '조명, 전기용품 포장, 생산직원 모집'}, {'j.name': '[강추] + 삼성전자 에어컨 설치파트 전문기사모집+배우며일하실 신입환영 +주기사가능'}, {'j.name': '조리찬담당 (국,볶음,무침종류, 전처리) 월250만 10시-19시 주5일스케줄근무'}, {'j.name': '농촌전원주택 조경및 소규모 텃밭 관리직원구함 (외국인도 가능)'}, {'j.name': '주방 (홀가능하신 분 환영) (주5일근무)'}, {'j.name': '한우 농장 <긴급 구인>'}, {'j.name': '생산직 모집 기술배우실분 초보가능 수습3개월후 급여인상'}, {'j.name': '(돈까스제조공장)생산직원 월270 경력우대'}, {'j.name': '단순현장직(족발가공,손질) 주5일근무 남자는숙소제공가능'}, {'j.name': '산후조리원 에서 주방조리사 월270만이상 / 주방보조 월235만 좋은조건으로 모집합니다.'}]

> Finished chain.


'죄송합니다만, 제가 제공받은 정보에는 시급이 1만원이 넘는 직업에 대한 정보가 없습니다.'

In [12]:
chain.run("양갈비 맛집을 추천해줘")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (r:Restaurant)-[:hasCategory]->(c:Category) WHERE c.name = '양갈비' RETURN r.name, r.address
Full Context:
[{'r.name': '이가네양갈비 판교점', 'r.address': '경기 성남시 분당구 판교역로 145'}]

> Finished chain.


'이가네양갈비 판교점을 추천드립니다. 주소는 경기 성남시 분당구 판교역로 145입니다.'

In [38]:
chain.run("가격이 30000에서 40000원 사이의 음식메뉴 3가지만 추천해줘")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (r:Restaurant)-[:SERVES]->(m:Menu) WHERE m.price >= 30000 AND m.price <= 40000 RETURN m.name LIMIT 3
Full Context:
[{'m.name': '스모크드 뽈뽀스테이크'}, {'m.name': '다이호시 카이센동'}, {'m.name': '사시미 모리아와세 1인'}]

> Finished chain.


'스모크드 뽈뽀스테이크, 다이호시 카이센동, 사시미 모리아와세 1인을 추천드립니다. 이 세 가지 메뉴는 가격이 30000에서 40000원 사이에 있습니다.'

In [7]:
# chain.run("가격이 3만원대의 메뉴를 파는 맛집 추천해줘")
chain.run("Please recommend a good restaurant that sells menus that cost around 30,000 won")



> Entering new GraphCypherQAChain chain...
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Generated Cypher:
MATCH (r:Restaurant)-[:SERVES]->(m:Menu) WHERE m.price = 30000 RETURN r.name, r.address, r.phone, r.operation
Full Context:
[{'r.name': '다이호시', 'r.address': '경기 성남시 분당구 판교역로10번길 6 1층 다이호시', 'r.phone': '0507-1425-8322', 'r.operation': '12:00에 영업 시작'}, {'r.name': '다이호시', 'r.address': '경기 성남시 분당구 판교역로10번길 6 1층 다이호시', 'r.phone': '0507-1425-8322', 'r.operation': '12:00에 영업 시작'}, {'r.name': '사쿠라테이엔 판교본점', 'r.address': '경기 성남시 분당구 판교역로192번길 16', 'r.phone': '0507-1353-6234', 'r.operation': '17:00에 영업 시작'}, {'r.name': '사쿠라테이엔 판교본점', 'r.address': '경기 성남시 분당구 판교역로192번길 16', 'r.phone': '0507-1353-6234', 'r.operation': '17:00에 영업 시작'}, {'r.name': '도아들', 'r.address': '경기 성남시 분당구 분당로53번길 15 1층 116호~119호', 'r.phone': '0507-1492-2065', 'r.operation': '21:00에 라스트오더'}, {'r.n

"I'm sorry, but the provided information does not include details about the menu prices at the restaurants."